# Butterfly Strategy Assesment

@Author: Daniel Rodríguez Delgado

@Date: 02/09/2019

In [2]:
from math import *
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew, pearsonr, norm, t
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


## 1. Pseudo-random number generators

In [3]:
def congru(seed, m=((2**31)-1), a=16807, b=0):
        y=(a*seed+b)%m
        u=y/m
        return(u, y)
    
def boxmuller(seed, mu=0, sigma=1):
    u1, seed=congru(seed)    #Uniform inputs
    u2, seed=congru(seed)
    
    x=sqrt(-2*log(u1))*cos(2*pi*u2)*sigma+mu  #Gaussian numbers generation
    y=sqrt(-2*log(u1))*sin(2*pi*u2)*sigma+mu
    
    antx=sqrt(-2*log(1-u1))*cos(2*pi*(1-u2))*sigma+mu  #Antithetic numbers generation
    anty=sqrt(-2*log(1-u1))*sin(2*pi*(1-u2))*sigma+mu
    return(x, y, u1, u2, antx, anty, seed)


def marsaglia(seed, mu=0, sigma=1, mode='normal'):
    
    while True:
        if mode=='normal':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*u1-1
            v2=2*u2-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        if mode=='antithetic':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*(1-u1)-1
            v2=2*(1-u2)-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        
    return(x1,x2,seed)

# Gamma distribution random generator
def gamma (p, a, seed):
    v=[]
    for i in range(p):
        u, seed= congru(seed)
        v.append(log(u))
    x=-sum(v)/a
    return(x, seed)


# Chi^2 distribution random generator
def chisq (n, seed):
    if (n%2==0):
        g, seed=gamma(int(n/2), 1/2, seed)
        return(g, seed)
    else:
        g, seed=gamma(int((n-1)/2), 1/2, seed)
        cos1, sen1, u1, u2, antx, anty, seed = boxmuller(seed)
        x=g+sen1**2
    return(x, seed)

# t-student distribution random generator
def tstudent (n, seed):
    cos1, sen1, u1, u2, antx, anty, seed=boxmuller(seed)
    xchi, seed=chisq(n, seed)
    x=sen1/sqrt(xchi/n)
    return(x, seed)


## 2. Initial Data

In [14]:
# Seed for Box-Müller method
seed=123456789
# Number of scenarios
N=1000
# Underlying price at the begining
S0=14
# First long call's Strike price
Sk1=14
# Second long call's Strike price
Sk2=16
# 2 Short calls' Strike price
Sk3=15

# Yearly and continous return
ry=0.03
rc=log(1+ry)

# Yearly and continuos discount rate
dy=0
dc=log(1+dy)

# Volatility (Sigma)
sig=0.3
#Period
d=12
T=1
dt=T/d

In [15]:
tM=np.empty((0,12))
payoff12c1=[]
payoff12c2=[]
payoff12p=[]
for i in range(N):
    tm=[]
    for j in range(d):
        if (j<1):
            
            S=S0*(1+ry-dc)**dt+sig*S0*np.sqrt(dt)*y
            tm.append(S)
        else:
            x, y, u1, u2, antx, anty, seed = boxmuller(seed, mu=0, sigma=1)
            S=S*(1+ry-dc)**dt+sig*S*np.sqrt(dt)*y
            tm.append(S)
    tm=np.array(tm)
    tM=np.vstack((tM,tm))    
    
    
    payoff12c1.append(max(np.mean(tm)-Sk1,0))
    payoff12c2.append(max(np.mean(tm)-Sk2,0))
    payoff12p.append(max(Sk3-np.mean(tm),0))

# Pay-off 12 months metrics for call option Strike=14
po12AVGc1=np.mean(payoff12c1)
po12VARc1=np.var(payoff12c1, ddof=1)
po12ACCc1=t.ppf(1-0.05/2,len(tM))*np.sqrt(po12VARc1/N)

val12AVGc1=po12AVGc1*np.exp(-rc*T)
val12VARc1=po12VARc1*np.exp(-2*rc*T)
val12ACCc1=t.ppf(1-0.05/2,len(tM))*np.sqrt(val12VARc1/N)

index=['mean', 'variance', 'accuracy']
metricsc1=pd.DataFrame({
                      'Pay-off': np.array([po12AVGc1, po12VARc1, po12ACCc1]),
                      'Value': np.array([val12AVGc1, val12VARc1, val12ACCc1])
                    }, index=index)

# Pay-off 12 months metrics for call option Strike=14
po12AVGc2=np.mean(payoff12c2)
po12VARc2=np.var(payoff12c2, ddof=1)
po12ACCc2=t.ppf(1-0.05/2,len(tM))*np.sqrt(po12VARc2/N)

val12AVGc2=po12AVGc2*np.exp(-rc*T)
val12VARc2=po12VARc2*np.exp(-2*rc*T)
val12ACCc2=t.ppf(1-0.05/2,len(tM))*np.sqrt(val12VARc2/N)

index=['mean', 'variance', 'accuracy']
metricsc2=pd.DataFrame({
                      'Pay-off': np.array([po12AVGc2, po12VARc2, po12ACCc2]),
                      'Value': np.array([val12AVGc2, val12VARc2, val12ACCc2])
                    }, index=index)


# Pay-off 12 months metrics for put option
po12AVGp=2*np.mean(payoff12p)
po12VARp=np.var(payoff12p, ddof=1)
po12ACCp=t.ppf(1-0.05/2,len(tM))*np.sqrt(po12VARp/N)

val12AVGp=po12AVGp*np.exp(-rc*T)
val12VARp=po12VARp*np.exp(-2*rc*T)
val12ACCp=t.ppf(1-0.05/2,len(tM))*np.sqrt(val12VARp/N)

index=['mean', 'variance', 'accuracy']
metricsp=pd.DataFrame({
                      'Pay-off': np.array([po12AVGp, po12VARp, po12ACCp]),
                      'Value': np.array([val12AVGp, val12VARp, val12ACCp])
                    }, index=index)

print('')
print(color.BOLD + 'Metrics for call option Sk=14:')
display( metricsc1)
print('')
print(color.BOLD + 'Metrics for call option Sk=16:')
display( metricsc2)
print('')
print(color.BOLD + 'Metrics for put option Sk=15:')
display(metricsp)    


Metrics for call option Sk=14:


,Pay-off,Value
mean,1.249563,1.213168
variance,3.442186,3.244590
accuracy,0.115131,0.111777



Metrics for call option Sk=16:


,Pay-off,Value
mean,0.485561,0.471419
variance,1.456899,1.373267
accuracy,0.074901,0.072720



Metrics for put option Sk=15:


,Pay-off,Value
mean,2.837770,2.755117
variance,2.500386,2.356854
accuracy,0.098125,0.095267
